In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
class NN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 50)
        self.out = nn.Linear(50, num_classes)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        out = F.softmax(self.out(x))
        return out


In [ ]:
input_size = 784
num_classes = 10
batch_size = 64
lr = 0.001
num_epoch = 3

In [ ]:
load_train = datasets.MNIST(root='datasets/', train=True, transform=transforms.ToTensor())
load_test = datasets.MNIST(root='datasets/', train=False, transform=transforms.ToTensor())

train = DataLoader(load_train, batch_size, shuffle=True)
test = DataLoader(load_test, batch_size, shuffle=True)

data, label = load_test[23]
print(label)
data

In [65]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = NN(input_size, num_classes).to(device)

optimizer = optim.Adam(model.parameters(), lr)
criterion = nn.CrossEntropyLoss()

for epoch in range(num_epoch):
    for batch_idx, (data, label) in enumerate(tqdm(train)):
        
        data = data.to(device=device)
        label = label.to(device=device)

        data = data.reshape(data.shape[0], -1)

        y = model(data)
        loss = criterion(y, label)

        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()

  0%|          | 0/938 [00:00<?, ?it/s]/tmp/ipykernel_31212/1115388479.py:11: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  out = F.softmax(self.out(x))
100%|██████████| 938/938 [00:10<00:00, 86.42it/s]


In [63]:
def check_accuracy(data, model):
    
    num_samples = 0
    num_correct = 0
    model.eval()

    with torch.no_grad():
        for x, label in data:

            x = x.to(device)
            label = label.to(device)

            x = x.reshape(x.shape[0], -1)
            y = model(x)

            _, predictions = y.max(1)

            num_correct += (predictions == label).sum()
            num_samples += predictions.shape[0]

    accuracy = num_correct / num_samples * 100
    print(float(accuracy))

In [64]:
check_accuracy(test, model)
check_accuracy(train, model)

/tmp/ipykernel_31212/1115388479.py:11: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  out = F.softmax(self.out(x))


95.0199966430664
95.72166442871094
